In [1]:
import sys
import time
import os.path
from glob import glob
from datetime import datetime, timedelta

# data tools
import h5py
import numpy as np
import numba as nb
import pandas as pd
import netCDF4 as nc

# custom tools
sys.path.insert(0, '/glade/u/home/ksha/PUBLISH/WFRT-PP-DEV/')

from namelist_casper import * 

#### SL indices

* `IxIy_unique`: a 2-d array of `(indx, indy)`; it extracts grid points that are needed for AnEn-SL

* `SL_xy`: a tuple of 12 arrays; one array per month. Each array has `(2574, 20)` elements. `2574` is the total number of BC grid points; `20` represents a BC grid point and its 19 SLs. Array values are indics of `IxIy_unique`

* NA West grids -- (`IxIy_unique`) --> unique grids for AnEn -- (`SL_xy`) --> SLs, multiple BC grids can have the same SLs. 

In [2]:
# supplemental locations
SL_xy_dict = {}
with h5py.File(save_dir+'SL20_d4_unique.hdf', 'r') as h5io:
    IxIy_unique = h5io['unique_inds'][...]
    for i in range(12):
        temp = h5io['mon_{}_inds'.format(i)][...]
        temp = temp.astype(int)
        SL_xy_dict['{}'.format(i)] = temp
IxIy_unique = IxIy_unique.astype(int)
SL_xy = tuple(SL_xy_dict.values())

In [3]:
len(SL_xy)

12

In [4]:
IxIy_unique.shape

(7641, 2)

In [5]:
year = 2000
lead = 2

with h5py.File(REFCST_dir+'En_mean_APCP_{}.hdf'.format(year), 'r') as h5io:
    apcp_temp = h5io['base_mean'][:, lead, ...][..., IxIy_unique[:, 0], IxIy_unique[:, 1]]
with h5py.File(REFCST_dir+'En_mean_PWAT_{}.hdf'.format(year), 'r') as h5io:
    pwat_temp = h5io['base_mean'][:, lead, ...][..., IxIy_unique[:, 0], IxIy_unique[:, 1]]

In [6]:
apcp_temp.shape

(366, 7641)

In [7]:
SL_xy[0].shape

(2574, 20)

In [8]:
SL_xy[0][0, :]

array([1626, 2180,  807, 1415, 1268, 2118, 2721, 3547, 2583, 1022,  533,
        922, 1987,  357, 1857, 1229,  437, 3886, 4470, 1694])

**AnEn mean** --> mapped by IxIy_unique --> Save as Zarr per year per lead time

### Test on Zarr

In [9]:
import zarr

In [10]:
zarr.save('test.zarr', apcp_temp)

In [11]:
np.save('test.npy', apcp_temp)

In [12]:
test_zarr = zarr.load('test.zarr')

### Scripts

This notebook is now `En_mean_compress.py`